In [1]:
import numpy as nm
import pandas as pa
import re
dataSetPath = r"tatoeba-dataset_Edited712k.tsv"

C:\Users\Yazılım alanı\AppData\Local\Temp\ipykernel_3244\3075349580.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pa


In [2]:
# Veri setinin içeri aktarılması:
dataEng = []
dataTR = []
with open(dataSetPath, 'r', encoding='utf8') as file:
    for line in file:
        parts = line.split('\t')
        dataEng.append(parts[1])
        dataTR.append(parts[3])
print(f"İçeri aktarılan cümle sayısı: {len(dataTR)}")

İçeri aktarılan cümle sayısı: 712143


In [3]:
# Verinin sıralanması ve uzun cümlelerin veri setinden çıkartılması:

# Verinin kelîme uzunluğuna göre sıralanması:
data = {"TR": dataTR, "ENG": dataEng}
dfData = pa.DataFrame(data)
dfData["len"] = dfData["ENG"].apply(lambda x: len(x.split()))
dfData.sort_values("len", ascending = True, inplace = True)

# Veri seti kısa metînlerden oluşuyor; fakat 190 kelîme uzunluğunda metînler de var. Metîn uzunluğunun sâbit olması lazım
# 12 kelîme seçildiğinde verilerin %95'inden fazlasını alabiliyoruz:
dfData = dfData[dfData["len"] <= 12]

dfData.drop("len", inplace = True, axis = 1)

In [4]:
# Fazla boşlukların, yeni satır karakterlerin cümlelerden çıkartılması:
dfData["ENG"] = dfData["ENG"].str.strip()
dfData["TR"] = dfData["TR"].str.strip()

In [5]:
# Küçük harfe çevirme:
# Özel isimler de küçük harfe çevriliyor;
# bunu önlemek için bu işlemden evvel özel isimleri NER (Named Entity Recognition) ile işâretlemek iyi bir fikir olabilir;
# fakat şu an için bunu yapmamız zâten karmaşık olan işi daha karmaşık hâle getirir.
dfData["ENG"] = dfData["ENG"].apply(lambda x: x.lower())
lowerTR = lambda x: x.replace('I', 'ı').lower()
dfData["TR"] = dfData["TR"].apply(lowerTR)

In [6]:
# Türkçe çevirilerde sıklıkla rastlanan bir yazım hatâsı: Allahın -> Allah'ın şeklinde düzeltilmeli

pattern = re.compile(r"Allah\w+")
dfData.reset_index(drop=True, inplace = True)
foundPat = dfData["TR"].apply(lambda x: re.search(pattern, x) is not None)
for i in range(0, len(foundPat)):
    if(foundPat[i]):
        dfData["TR"].iloc[i] = dfData["TR"].iloc[i].replace(r"Allah", r"Allah'")

In [7]:
# Veri arttırımı (data augmentation)
# Veri içerisinde "he's", "I'm", "doesn't" gibi kısaltmalar var.
# Bunların gerçekte "he is", "I am", "does not" olduğunun öğrenilmesi için
# bunların bir kısmının uzun hâlini de veri olarak veri setine eklemek iyi bir fikir olabilir.
# "he's" kelîmesinin anlamının "he is" olduğu daha iyi modellenebilir.
# Veri arttırımı için örüntüler:
pat = re.compile(r"\w*'re")# 're -> (you | they | we) are
pat2 = re.compile(r"\w{1,4}'ve")# 've -> (I | you | they | we) have
pat3 = re.compile(r"i'm")# i'm -> I am
pat4 = re.compile(r"\w{1, 4}'ll")# (I | you | we | they | he | she | it)'ll -> will
pat5 = re.compile(r"doesn't")# doesn't -> does not
pat6 = re.compile(r"isn't")# isn't -> is not
pat7 = re.compile(r"aren't")# aren't -> are not
pat8 = re.compile(r"won't")# won't -> will not
pat9 = re.compile(r"couldn't")# couldn't -> could not
pat10 = re.compile(r"wouldn't")# wouldn't -> would not
pat11 = re.compile(r"wasn't")# wasn't -> was not
pat12 = re.compile(r"were't")# were't -> were not
pat13 = re.compile(r"haven't")# haven't -> have not
pat14 = re.compile(r"hasn't")# hasn't -> has not
pat15= re.compile(r"hadn't")# hadn't -> had not
pat16 = re.compile(r"\w{1,4}'d")# [I | you | we | they | he | she | it]'d -> .. had
pat17 = re.compile(r"don't")# don't -> do not
pat18 = re.compile(r"\w{1,3}'s been")# 's been -> 's has been
pat19 = re.compile(r"\w{1,3}'s (?!been)")# (he | she | it)'s -> .. is (been içermeyen) 

# 's takısı pek çok anlama gelebilmektedir. Sâhiplik olan ek ile karışmaması için sadece zamirlere uygulanmalı; fakat
# zamire gelen 's takısı 'has' yardımcı fiiline de işâret edebilmektedir.
# Bunu ayırt etmek için sonrasında 'been' ifâdesinin gelip, gelmediğine bakmalıyız:
# he's a doctor -> 'be' fiili -> he is
# he's been working for 5 hour -> 'has' -> he has
# BU OLMAZ: pat8 = re.compile(r"she's|it's|he's")# he's, He's, she's, She's, It's, it's -> .. is

In [8]:
# Verilen örüntüyü verilen veri içerisinde arayıp, belirtilen dönüşümlere göre veri üretimi yapan fonksiyon:
def applyPattern(dataAsDataFrame, pattern, fromThatToApplyThisDict, applyToOdd: bool = False, applyToEven: bool = True):
    """
    Verilen veri DataFrame formatında olmalı
    İngilizce veriler "ENG" sütununda, Türkçe veriler "TR" sütununda yer almalı
    Geriye arttırılan veriler liste olarak döndürülür; DataFrame'e kaydedilmez
    Geriye döndürülen listenin her elemanı bir sözlüktür; bu sözlükte "ENG" anahtarında İngilizce cümle, "TR" anahtarında
    Türkçe cümle yer almaktadır
    Verilerin indisleri 0, 1, 2, 3... şeklinde sıralı şekilde olmalıdır
    """
    newData = []
    foundPat = dataAsDataFrame["ENG"].apply(lambda x: re.search(pattern, x) is not None)
    for i in range(0, len(foundPat)):#dataAsDataFrame["ENG"][foundPat == True]:
        if not foundPat.iloc[i]:
            continue
        even = (i % 2 == 0)
        if (applyToOdd and not even) or (applyToEven and even):
            sent = dataAsDataFrame["ENG"].iloc[i]
            for key in fromThatToApplyThisDict:
                sent = sent.replace(key, fromThatToApplyThisDict[key])
            engTRDict = {}
            engTRDict["ENG"] = sent
            engTRDict["TR"] = dataAsDataFrame["TR"].iloc[i]
            newData.append(engTRDict)
    return newData

In [9]:
# Veri arttırımı için gerekli dönüşüm verileri:
patternData = {"you're" : "you are", "they're" : "they are", "we're" : "we are"}
patData2 = {"i've": "i have", "you've": "you have", "they've": "they have", "we've": "we have"}
patData3 = {"i'm": "i am"}
patData4 = {"i'll": "i will", "you'll": "you will", "he'll": "he will", "she'll": "she will", "it'll": "it will",
            "we'll": "we will", "they'll": "they will"}
patData5 = {"doesn't": "does not"}
patData6 = {"isn't": "is not"}
patData7 = {"aren't": "are not"}
patData8 = {"won't": "will not"}
patData9 = {"couldn't": "could not"}
patData10 = {"wouldn't": "would not"}
patData11 = {"wasn't": "was not"}
patData12 = {"weren't": "were not"}
patData13 = {"haven't": "have not"}
patData14 = {"hasn't": "has not"}
patData15 = {"hadn't": "had not"}
patData16 = {"i'd": "i had", "you'd": "you had", "we'd": "we had", "he'd": "he had", "she'd": "she had", "it'd": "it had",
            "they'd": "they had"}
patData17 = {"don't": "do not"}
patData18 = {"he's": "he has", "she's": "she has", "it's": "it has"}
patData19 = {"she's": "she is", "he's": "he is", "it's": "it is"}

# Belirtilen dönüşümler, ilgili örüntünün bulunduğu cümlelerin yarısına (çift indisli olanlarına) uygulanmaktadır:
patterns = [pat, pat2, pat3, pat4, pat5, pat6, pat7, pat8, pat9, pat10, pat11, pat12, pat13, pat14, pat15, pat16]
patAllData = [patternData, patData2, patData3, patData4, patData5, patData6, patData7, patData8, patData9, patData10,
             patData11, patData12, patData13, patData14, patData15, patData16]
allTogether = dict(zip(patterns, patAllData))

In [10]:
# Veri arttırımı: Hâzırladığımız örüntülerle 100 bine yakın yeni veri çıkartılabiliyor; bunu kısıtlamak için her birinden
# bin adet veri alınabilir
newData = []
for pattern, patternData in allTogether.items():
    produced = applyPattern(dfData, pattern, patternData)
    for i in range(0,len(produced)):
        newData.append(produced[i])
        if i == 1000:
            break
print(f"Eklenen yeni cümle sayısı: {len(newData)}")

Eklenen yeni cümle sayısı: 12892


In [11]:
# Yeni verilerin veri setine eklenmesi:
#print(f"Üretilen yeni cümle sayısı: {len(newData)}")
dfNew = pa.DataFrame(columns = ["ENG", "TR"], data = newData)
dfData = pa.concat([dfData, dfNew])
print(f"Veri seti nihâî boyutu: {len(dfData)}")

Veri seti nihâî boyutu: 703931


In [12]:
# Yeni verilerin hazineye eklenmesinden sonra verileri yeniden sıralama:
dfData["len"] = dfData["ENG"].apply(lambda x: len(x.split()))
dfData.sort_values("len", ascending = True, inplace = True)
dfData.drop("len", inplace = True, axis = 1)

In [14]:
# Verinin dışa aktarılması:
dfData.to_csv("cleaned703KSentences.csv", encoding = "utf8", index = False, sep = '\t')
# Cümlelerde virgül olabilir; bu sebeple sep parametresine '\t' gibi metîn içerisinde olmayan bir karakter verilmeli